### Decorators Application (Memoiation)
Let's go to fibonacci example

In [44]:
def fib(n):
    print ('Calculating fib({0})'.format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

When we run this, we see that it is quite inefficient, as the same Fibonacci numbers get calculated multiple times:

In [45]:
fib(6)

Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)


8

It would be better if we could somehow "store" these results, so if we have calculated `fib(4)` and `fib(3)` before, we could simply recall the these values when calculating `fib(5) = fib(4) + fib(3)` instead of recalculating them.

This concept of improving the efficiency of our code by caching pre-calculated values so they do not need to be re-calcualted every time, is called "memoization"

>We can approach this using a simple class and a dictionary that stores any Fibonacci number that's already been calculated:

In [50]:
class Fib:
    def __init__(self):
        self.cache = {1: 1, 2: 1}

    def fib(self, n):
        if n not in self.cache:
            print("Caculating fib({0})".format(n))
            self.cache[n] = self.fib(n - 1) + self.fib(n - 2)
        return self.cache[n]

In [51]:
# Create fibonacci object
f = Fib()

In [52]:
f.fib(6)

Caculating fib(6)
Caculating fib(5)
Caculating fib(4)
Caculating fib(3)


8

In [53]:
f.fib(7)

Caculating fib(7)


13

>Let's see how we could rewrite this using a closure:

In [57]:
def fib():
    cache = {1:1,2:1}
    
    def calc_fib(n):
        if n not in cache:
            print("Caculating fib({0})".format(n))
            cache[n] = calc_fib(n-1) + calc_fib(n-2)
        return cache[n]
    return calc_fib

In [61]:
f = fib()  # calc_fib function is stored in 'f' so f behaves like 'calc_fib'

In [62]:
f(10)

Caculating fib(10)
Caculating fib(9)
Caculating fib(8)
Caculating fib(7)
Caculating fib(6)
Caculating fib(5)
Caculating fib(4)
Caculating fib(3)


55

In [63]:
f(11)

Caculating fib(11)


89

Now let's see how we would implement this using a decorator:

In [64]:
from functools import wraps

def memoize_fib(fn):
    cache = dict()
    
    @wraps(fn)
    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    
    return inner

In [65]:
@memoize_fib
def fib(n):
    print ('Calculating fib({0})'.format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [66]:
fib(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


5

In [67]:
fib(6)

Calculating fib(6)


8

As you can see, we are hitting the cache when the values are available.

Now, we made our memoization decorator "hardcoded" to single argument functions - we could make it more generic.

For example, to handle an arbitrary number of positional arguments and keyword-only arguments we could do the following:

In [68]:
def memoize(fn):
    cache = dict()
    
    @wraps(fn)
    def inner(*args):
        if args not in cache:
            cache[args] = fn(*args)
        return cache[args]
    
    return inner

In [69]:
@memoize
def fib(n):
    print ('Calculating fib({0})'.format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [70]:
fib(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


5

In [71]:
fib(6)

Calculating fib(6)


8

Of course, with this rather generic memoization decorator we can memoize other functions too:

In [72]:
def fact(n):
    print('Calculating {0}!'.format(n))
    return 1 if n < 2 else n * fact(n-1)

In [73]:
fact(5)

Calculating 5!
Calculating 4!
Calculating 3!
Calculating 2!
Calculating 1!


120

In [74]:
fact(6)

Calculating 6!
Calculating 5!
Calculating 4!
Calculating 3!
Calculating 2!
Calculating 1!


720

>And memoizing it:

In [75]:
@memoize
def fact(n):
    print('Calculating {0}!'.format(n))
    return 1 if n < 2 else n * fact(n-1)

In [76]:
fact(5)

Calculating 5!
Calculating 4!
Calculating 3!
Calculating 2!
Calculating 1!


120

In [77]:
fact(6)

Calculating 6!


720

Our simple memoizer has a drawback however:
* the cache size is unbounded - probably not a good thing! In general we want to limit the cache to a certain number of entries, balancing computational efficiency vs memory utilization.
* we are not handling **kwargs


Memoization is such a common thing to do that Python actually has a memoization decorator built for us!

It's in the, you guessed it, **functools** module, and is called **lru_cache** and is going to be quite a bit more efficient compared to the rudimentary memoization example we did above.

[LRU Cache = Least Recently Used caching: since the cache is not unlimited, at some point cached entries need to be discarded, and the least recently used entries are discarded first]

In [78]:
from functools import lru_cache
@lru_cache()
def fact(n):
    print("Calculating fact({0})".format(n))
    return 1 if n < 2 else n * fact(n-1)

In [79]:
fact(5)

Calculating fact(5)
Calculating fact(4)
Calculating fact(3)
Calculating fact(2)
Calculating fact(1)


120

In [81]:
fact(4)

24

You may have noticed that the `lru_cache` decorator was implemented using `()` - we'll see more on this later, but that's because decorators can themselves have parameters (beyond the function being decorated).

One of the arguments to the `lru_cache` decorator is the size of the cache - it defaults to 128 items, but we can easily change this - for performance reasons use powers of 2 for the cache size (or None for unbounded cache):

In [82]:
@lru_cache(maxsize=8)
def fib(n):
    print("Calculating fib({0})".format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [85]:
fib(10)

55

In [86]:
fib(20)

Calculating fib(20)
Calculating fib(19)
Calculating fib(18)
Calculating fib(17)
Calculating fib(16)
Calculating fib(15)


6765

In [87]:
fib(6)

Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


8

Here we can see that when we fib(10) was calculated earlier, and lru_cache stored latest 8 items, further when we calculateed fib(20) then it stored latest 8 items and erased everything else, and when we again calculated fib(6) then it has to recalculate all the valued required in fib(6) again because these items were already deleted.

You'll not how Python had to recalculate `fib` for `10, 9,` etc. This is because the cache can only contain 10 items, so when we calculated `fib(20)`, it stored fib for `20, 19, ..., 11` (10 items) and therefore the oldest items fib `10, 9, ..., 1` were removed from the cache to make space.